In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
import datetime
import regex
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer


In [ ]:

# Define Banknames and common versions to replace with "Bank later"
banknames = ["C24 Bank","deutsche ing (diba)","bank c24","C24","Deutsche Bank","ING Deutschland","bank ing-diba","ing diba bank","ing-diba bank","ing.diba","bank ing","ingdiba","ing-diba","ing diba","ing bank","ing  diba","n26 bank","bank n26"
,"N26","Sparda-Bank BW","Sparda-Bank Südwest eG","Volksbank","Commerzbank","dkb bank","bank dkb","DKB","HypoVereinsbank","Hypo","Bunq","bunq bank"
,"Postbank","PSD Braunschweig","PDF Rhein Ruhr","PSD Karlsruhe","PSD berlin brandenburg","Sparda-Bank Hamburg","Sparda-Bank München","sparda- banken","Sparda-Bank Nürnberg"
,"Sparkasse Hannover","Volksbank Mittelhessen","volksbank Berlin","Volksbank Hamburg","Sparkasse Köln bonn",
"sparkasse bank","Sparkasse","sparkasse","Sparda-Bank",
"spardap","sparda bank","sparda banken","psd bank","psd","targobank","revolut","targo bank","tomorrow","vivid money","vividmoney","vivid-money","vivid","fidor bank","fidor","comdirect","norisbank","santander","noris bank","bunq","Fidorbank","fidor bank"
" diba ","dkb"," Diba "," diba","(diba)","diba ","bank deutschen bank","deutsche Bank","deutschen bank"," ING ","Bankbank","bank "]
stop_words = stopwords.words("german") 



In [ ]:
#Get the file path

import glob
datafilepaths = glob.glob("MorerecentData/*")

In [ ]:
#Write Stopwords to excel for appendix
stop_words= pd.DataFrame(data=stop_words)
with pd.ExcelWriter('stopwords.xlsx') as writer:  
    stop_words.to_excel(writer, sheet_name='stopwords',engine="xlsxwriter")

In [ ]:
#Get customerreview data and create a Dataframe object
customerevaluations = pd.DataFrame(columns=['company','date','headline','body','rating'])

for files in datafilepaths:
    customerevaluation = pd.read_csv(files)
    customerevaluations = pd.concat([customerevaluations,customerevaluation],ignore_index=True)

customerevaluations["date"] = pd.to_datetime(customerevaluations["date"])
customerevaluations.drop_duplicates(["headline","body","rating"])
customerevaluations["length of review"] = customerevaluations["body"].str.len()

#customerevaluations['body'] = customerevaluations['body'].map(lambda x: x.lower())
customerevaluations["rating"] = pd.to_numeric(customerevaluations['rating'])

In [ ]:
#Generate some descriptives
summarytable = customerevaluations[["rating","company","length of review"]].groupby(["company"]).describe().round(decimals=2)
summarybyrating= customerevaluations[["rating","length of review"]].groupby("rating").describe().round(decimals=2)
customerevaluations['company'] = customerevaluations['company'].map(lambda x: re.sub('ING Neu2', 'ING', x))
customerevaluations['company'] = customerevaluations['company'].map(lambda x: re.sub('ING Neu', 'ING', x))
"""
with pd.ExcelWriter('summary tables.xlsx') as writer:  
    summarytable.to_excel(writer, sheet_name='summarybybank')
    summarybyrating.to_excel(writer, sheet_name='summarybyrating')
"""


In [ ]:
# Do preprocessing
customerevaluations["originalbody"] = customerevaluations['body']
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('&nt', '', x))

for word in banknames:
    customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(word.lower(),'Bank ', x.lower()))
'''
for word in stop_words:
    #customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(" "+word.lower()+" ", ' ', x))
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('&', '', x))
'''
customerevaluations['word count'] = customerevaluations['originalbody'].map(lambda x: len(x.strip().split(" ")))






In [ ]:
#filter out spam comments
searchfor = ['mois',"Mois","MOIS","keller","Keller","nahuiiiiiiiiiiii","lakakakakslskdnwkskd","einfach aus prinzip nahuyyyyyyy",
"nahui","oh yeah nahui","nahuiiiii","kellwerwärter","schnelllgmarceletkevin","83 jahre alt und nach schlaganfall pflegebedürftig","mühlhshabbshsbshyjabvsuabsbsjabasjss","de54100110012621571864","huens","ich kunde der deutschen bank (odenkirchener str. 43, 41236 mönchengladbach)","bernlöhr66538","inshallah","ist ja ekelhaft","drecksverein.geht den bach runter.","leute geht niemals zur Bank der gröste müll","gröster müll der welt.",".yyyyyyyyyyyyyyyyyy","oaschlöcher","ja sie wollen, dass ich falle lele"
,"gelber oasch nahoi","ya kelb ihr seid einfach lächerlich","jetzt werden önkel gefickt najui","alles neider","richtige lipp lipps die gehören gehauen","wiiixxxxxxxerrrr","erika","wenn koi luscht hen koi zeit hen wedder bleed ish en shineesischer saddelit uf erd stertze dud, konschde deene wadde, un wadde, un wadde, un..., bassiert nix. hen ja fielleischt nerffe","xxxxxxxxxcccccxcxc","einfach aus prinzip nahuyyyyyyy"]
customerevaluations = customerevaluations[~customerevaluations.body.str.contains('|'.join(searchfor))]
customerevaluations['date'] = customerevaluations['date'].apply(lambda a: pd.to_datetime(a).date()) 
customerevaluations = customerevaluations[(customerevaluations["date"]< (datetime.date(2022,4,16)))]


In [ ]:
#Substitute Euros
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('\d{1,},\d{1,}.€', 'Betrag', x))
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('\d{1,}.euro', 'Betrag', x))

In [ ]:
#Descriptives rating
customerevaluations.groupby("rating").mean()

In [ ]:
from bertopic import BERTopic

In [ ]:
#Set variables for model
vectorizer_model = CountVectorizer(stop_words=stop_words)
customerevaluations.reset_index(drop=True, inplace=True)
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = sentence_model.encode(customerevaluations["body"], show_progress_bar=False)

In [ ]:
#Generate model
customerevaluations.reset_index(drop=True, inplace=True)
topic_model = BERTopic(language="german",calculate_probabilities=False,top_n_words=10,vectorizer_model=vectorizer_model,diversity=0.3,min_topic_size=40)
topics, probs = topic_model.fit_transform(customerevaluations["body"],embeddings)


In [ ]:
#Get model summary
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} documents have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} documents are {topic_freq['Topic'].shape[0]-1} topics")

In [ ]:
#Reduce model to 80
topics, probs= topic_model.reduce_topics(customerevaluations["body"], topics, nr_topics=80)
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10,top_n_topics=60)

In [ ]:
customerevaluations["topic"] = topics

In [ ]:
#Extraxt relevant word for each topic
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
topicsAndTopWords = []
#for i in range(1,len(topic_freq["Topic"])):
for i in range(0,len(topic_freq["Topic"])-1):
    t = topic_model.get_topic(i)
    words = ""

    for w in range(0,len(t)-1):
        words = words + "_" + t[w][0] 
    topicsAndTopWords.append(words)

topicsAndTopWords = pd.DataFrame({"top words": topicsAndTopWords})



In [ ]:
topic_model.visualize_barchart(n_words=10,top_n_topics=60)

In [ ]:

with pd.ExcelWriter('15.06NRautoreduced to 100.xlsx') as writer:  
    customerevaluations.to_excel(writer, sheet_name='evaluations',engine="xlsxwriter")
    topicsAndTopWords.to_excel(writer, sheet_name='topics',engine="xlsxwriter")
    #reducedtopicsAndTopWords.to_excel(writer, sheet_name='reduced_topics')

In [ ]:

'''#len(reduction[0]) #to reduce topics then before doing everything with reduced model.
#topic_model.save("Mymodel15topnword")
with pd.ExcelWriter('sentimentanalysis.xlsx') as writer:  
    diccdf.to_excel(writer, sheet_name='Wordlist')
    
'''

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
#topic_model.save("15.06NRautoreduced to 100")

In [ ]:
topic_modeltestreload = BERTopic.load("15.06NRautoreduced to 50")

In [ ]:
topic_modeltestreload.visualize_barchart()